In [269]:
import pandas as pd
from plotting import CandlePlot
pd.set_option('display.max_columns', None)

In [270]:
df_ma_res = pd.read_pickle("../data/ma_res_2023-12-04.pkl")
df_ma_trades = pd.read_pickle("../data/ma_trades_2023-12-04.pkl")

In [271]:
df_ma_trades.head()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,MA__40,MA__200,MA__10,MA__80,MA__50,MA__20,MA__150,MA__120,MA__30,DELTA,DELTA_PREV,TRADE,DIFF,GAIN,granularity,pair,GAIN_C,ma_l,ma_s,cross
0,2023-10-09 11:00:00+00:00,960,1.05284,1.05347,1.05281,1.05340,1.05277,1.05340,1.05273,1.05332,1.05291,1.05354,1.05289,1.05348,1.054488,1.054700,1.053383,1.056074,1.054893,1.053323,1.055239,1.055435,1.053973,0.000060,-0.000030,1,-0.00016,-1.6,M15,EUR_USD,-1.6,MA__20,MA__10,MA__10_MA__20
1,2023-10-09 11:30:00+00:00,671,1.05358,1.05404,1.05314,1.05324,1.05349,1.05397,1.05306,1.05317,1.05366,1.05412,1.05322,1.05332,1.054347,1.054722,1.053210,1.055931,1.054786,1.053229,1.055219,1.055428,1.053850,-0.000019,0.000044,-1,0.00190,-19.0,M15,EUR_USD,-20.6,MA__20,MA__10,MA__10_MA__20
2,2023-10-09 13:30:00+00:00,2158,1.05465,1.05536,1.05440,1.05514,1.05458,1.05529,1.05433,1.05506,1.05472,1.05543,1.05448,1.05521,1.053859,1.054801,1.053514,1.055439,1.054293,1.053448,1.055154,1.055406,1.053387,0.000066,-0.000031,1,-0.00142,-14.2,M15,EUR_USD,-34.8,MA__20,MA__10,MA__10_MA__20
3,2023-10-09 17:00:00+00:00,732,1.05358,1.05388,1.05352,1.05372,1.05350,1.05380,1.05345,1.05365,1.05365,1.05396,1.05360,1.05380,1.053676,1.054959,1.054133,1.054659,1.053889,1.054184,1.055157,1.055299,1.053794,-0.000051,0.000082,-1,0.00292,-29.2,M15,EUR_USD,-64.0,MA__20,MA__10,MA__10_MA__20
4,2023-10-09 18:15:00+00:00,674,1.05672,1.05674,1.05650,1.05664,1.05664,1.05666,1.05643,1.05657,1.05679,1.05682,1.05658,1.05672,1.054079,1.055030,1.054852,1.054699,1.053956,1.054788,1.055214,1.055303,1.054305,0.000064,-0.000103,1,0.00024,2.4,M15,EUR_USD,-61.6,MA__20,MA__10,MA__10_MA__20


In [272]:
df_ma_trades.granularity.unique()

array(['M15', 'M30', 'H1', 'H4'], dtype=object)

In [273]:
def filter_data_by_granularity(df_ma_res, df_ma_trades, granularity):
    """
    Filter DataFrame based on granularity and create copies.

    Parameters:
    - df_ma_res (pd.DataFrame): DataFrame for macro results.
    - df_ma_trades (pd.DataFrame): DataFrame for trade results.
    - granularity (str): Granularity to filter on.

    Returns:
    pd.DataFrame, pd.DataFrame: Filtered DataFrames for macro results and trade results.

    Description:
    This function takes two DataFrames, df_ma_res and df_ma_trades, and filters them based on the provided granularity.
    It creates copies of the filtered DataFrames and returns them as df_res_filtered and df_trades_filtered, respectively.
    """
    df_res_filtered = df_ma_res[df_ma_res.granularity == granularity].copy()
    df_trades_filtered = df_ma_trades[df_ma_trades.granularity == granularity].copy()
    return df_res_filtered, df_trades_filtered

df_ma_res_m15, df_ma_trades_m15 = filter_data_by_granularity(df_ma_res, df_ma_trades, 'M15')
df_ma_res_m30, df_ma_trades_m30 = filter_data_by_granularity(df_ma_res, df_ma_trades, 'M30')
df_ma_res_h1, df_ma_trades_h1 = filter_data_by_granularity(df_ma_res, df_ma_trades, 'H1')
df_ma_res_h4, df_ma_trades_h4 = filter_data_by_granularity(df_ma_res, df_ma_trades, 'H4')


In [274]:
df_ma_res_m15.columns
df_ma_res_m30.columns
df_ma_res_h1.columns
df_ma_res_h4.columns

Index(['pair', 'num_trades', 'total_gain', 'mean_gain', 'min_gain', 'max_gain',
       'ma_l', 'ma_s', 'cross', 'granularity'],
      dtype='object')

In [275]:
def generate_cross_summary(df_ma_res, granularity):
    """
    Generate cross summary DataFrame based on granularity.

    Parameters:
    - df_ma_res (pd.DataFrame): DataFrame for macro results.
    - granularity (str): Granularity to filter on.

    Returns:
    pd.DataFrame: Cross summary DataFrame.

    Description:
    This function generates a cross summary DataFrame from the provided macro results DataFrame (df_ma_res)
    by filtering it based on the specified granularity. It creates a copy of the filtered DataFrame and extracts
    columns 'pair', 'number_of_trades', 'total_gain', and 'cross' to form the cross summary DataFrame.
    """
    df_res_filtered = df_ma_res[df_ma_res.granularity == granularity].copy()
    df_cross_summary = df_res_filtered[['pair', 'num_trades', 'total_gain', 'cross']].copy()
    return df_cross_summary

df_cross_summary_m15 = generate_cross_summary(df_ma_res, 'M15')
df_cross_summary_m30 = generate_cross_summary(df_ma_res, 'M30')
df_cross_summary_h1 = generate_cross_summary(df_ma_res, 'H1')
df_cross_summary_h4 = generate_cross_summary(df_ma_res, 'H4')


In [276]:
df_cross_summary_h4

,pair,num_trades,total_gain,cross
432,EUR_USD,213,-1430,MA__10_MA__20
433,EUR_USD,121,-1582,MA__10_MA__40
434,EUR_USD,112,-616,MA__20_MA__40
435,EUR_USD,128,-1030,MA__30_MA__40
436,EUR_USD,76,190,MA__10_MA__80
...,...,...,...,...
571,GBP_JPY,51,-5071,MA__10_MA__200
572,GBP_JPY,45,-5647,MA__20_MA__200
573,GBP_JPY,41,-6392,MA__30_MA__200
574,GBP_JPY,33,-3451,MA__40_MA__200


In [277]:
def group_and_sort(df):
    grouped_df = df.groupby(by="cross").sum()
    sorted_df = grouped_df.sort_values(by="cross", inplace=False, ascending=False)
    return sorted_df

df_cross_gr_m15 = group_and_sort(df_cross_summary_m15)
df_cross_gr_m30 = group_and_sort(df_cross_summary_m30)
df_cross_gr_h1 = group_and_sort(df_cross_summary_h1)
df_cross_gr_h4 = group_and_sort(df_cross_summary_h4)

In [278]:
df_cross_summary_h4

,pair,num_trades,total_gain,cross
432,EUR_USD,213,-1430,MA__10_MA__20
433,EUR_USD,121,-1582,MA__10_MA__40
434,EUR_USD,112,-616,MA__20_MA__40
435,EUR_USD,128,-1030,MA__30_MA__40
436,EUR_USD,76,190,MA__10_MA__80
...,...,...,...,...
571,GBP_JPY,51,-5071,MA__10_MA__200
572,GBP_JPY,45,-5647,MA__20_MA__200
573,GBP_JPY,41,-6392,MA__30_MA__200
574,GBP_JPY,33,-3451,MA__40_MA__200


In [279]:
def filter_ma_cross(df, cross_value):
    return df[df['cross'] == cross_value]

cross_value = "MA__40_MA__200"
temp = filter_ma_cross(df_ma_res_h4, cross_value)


In [280]:
temp[temp.total_gain > 0]

,pair,num_trades,total_gain,mean_gain,min_gain,max_gain,ma_l,ma_s,cross,granularity
454,EUR_USD,23,614,26,-303,628,MA__200,MA__40,MA__40_MA__200,H4
526,USD_JPY,21,2053,97,-497,1210,MA__200,MA__40,MA__40_MA__200,H4
550,GBP_USD,27,4,0,-446,772,MA__200,MA__40,MA__40_MA__200,H4


In [281]:
temp

,pair,num_trades,total_gain,mean_gain,min_gain,max_gain,ma_l,ma_s,cross,granularity
454,EUR_USD,23,614,26,-303,628,MA__200,MA__40,MA__40_MA__200,H4
478,EUR_GBP,36,-2033,-56,-152,171,MA__200,MA__40,MA__40_MA__200,H4
502,EUR_JPY,27,-1937,-71,-488,1133,MA__200,MA__40,MA__40_MA__200,H4
526,USD_JPY,21,2053,97,-497,1210,MA__200,MA__40,MA__40_MA__200,H4
550,GBP_USD,27,4,0,-446,772,MA__200,MA__40,MA__40_MA__200,H4
574,GBP_JPY,33,-3451,-104,-985,1675,MA__200,MA__40,MA__40_MA__200,H4


In [282]:
total_pairs = len(df_cross_summary_h4)

for c in df_cross_summary_h4['cross'].unique():
    temp = df_cross_summary_h4[df_cross_summary_h4['cross'] == c]
    pg = temp[temp['total_gain'] > 0].shape[0]
    print(f"{c} pg: {pg} percent gain: {(pg / total_pairs) * 100:.2f}%")


MA__10_MA__20 pg: 3 percent gain: 2.08%
MA__10_MA__40 pg: 2 percent gain: 1.39%
MA__20_MA__40 pg: 1 percent gain: 0.69%
MA__30_MA__40 pg: 2 percent gain: 1.39%
MA__10_MA__80 pg: 4 percent gain: 2.78%
MA__20_MA__80 pg: 3 percent gain: 2.08%
MA__30_MA__80 pg: 3 percent gain: 2.08%
MA__40_MA__80 pg: 4 percent gain: 2.78%
MA__50_MA__80 pg: 4 percent gain: 2.78%
MA__10_MA__120 pg: 3 percent gain: 2.08%
MA__20_MA__120 pg: 3 percent gain: 2.08%
MA__30_MA__120 pg: 3 percent gain: 2.08%
MA__40_MA__120 pg: 3 percent gain: 2.08%
MA__50_MA__120 pg: 3 percent gain: 2.08%
MA__10_MA__150 pg: 2 percent gain: 1.39%
MA__20_MA__150 pg: 3 percent gain: 2.08%
MA__30_MA__150 pg: 3 percent gain: 2.08%
MA__40_MA__150 pg: 3 percent gain: 2.08%
MA__50_MA__150 pg: 3 percent gain: 2.08%
MA__10_MA__200 pg: 3 percent gain: 2.08%
MA__20_MA__200 pg: 3 percent gain: 2.08%
MA__30_MA__200 pg: 3 percent gain: 2.08%
MA__40_MA__200 pg: 3 percent gain: 2.08%
MA__50_MA__200 pg: 2 percent gain: 1.39%


In [283]:
cross = "MA__40_MA__200"


df_MA_40_MA_200 = df_ma_res_h4[df_ma_res_h4.cross == cross].copy()

df_MA_40_MA_200.sort_values(by=['total_gain'], ascending=False, inplace=True)


In [284]:
df_MA_40_MA_200

,pair,num_trades,total_gain,mean_gain,min_gain,max_gain,ma_l,ma_s,cross,granularity
526,USD_JPY,21,2053,97,-497,1210,MA__200,MA__40,MA__40_MA__200,H4
454,EUR_USD,23,614,26,-303,628,MA__200,MA__40,MA__40_MA__200,H4
550,GBP_USD,27,4,0,-446,772,MA__200,MA__40,MA__40_MA__200,H4
502,EUR_JPY,27,-1937,-71,-488,1133,MA__200,MA__40,MA__40_MA__200,H4
478,EUR_GBP,36,-2033,-56,-152,171,MA__200,MA__40,MA__40_MA__200,H4
574,GBP_JPY,33,-3451,-104,-985,1675,MA__200,MA__40,MA__40_MA__200,H4


In [285]:
df_gains = df_ma_trades_h4[(df_ma_trades_h4.cross == cross) & (df_ma_trades_h4.pair == "USD_JPY")].copy()


In [286]:
df_gains.head(3)

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,MA__40,MA__200,MA__10,MA__80,MA__50,MA__20,MA__150,MA__120,MA__30,DELTA,DELTA_PREV,TRADE,DIFF,GAIN,granularity,pair,GAIN_C,ma_l,ma_s,cross
29834,2021-07-15 09:00:00+00:00,14509,109.749,110.090,109.730,110.067,109.742,110.084,109.723,110.060,109.756,110.097,109.736,110.074,110.236775,110.245160,110.1372,110.585375,110.35306,110.25760,110.454040,110.576525,110.176767,-0.008385,0.00746,-1,0.343,-34.3,H4,USD_JPY,-34.3,MA__200,MA__40,MA__40_MA__200
29835,2021-08-13 01:00:00+00:00,3634,110.420,110.427,110.371,110.410,110.414,110.420,110.364,110.404,110.427,110.434,110.377,110.417,110.203625,110.189915,110.4122,109.894962,109.97954,110.47765,109.998473,109.950983,110.379433,0.013710,-0.02839,1,-0.878,-87.8,H4,USD_JPY,-122.1,MA__200,MA__40,MA__40_MA__200
29836,2021-08-17 09:00:00+00:00,17464,109.306,109.558,109.174,109.532,109.300,109.551,109.164,109.525,109.313,109.565,109.184,109.539,110.105675,110.106015,109.3497,109.822050,110.06066,109.78740,109.930580,109.923617,110.038967,-0.000340,0.00966,-1,0.376,-37.6,H4,USD_JPY,-159.7,MA__200,MA__40,MA__40_MA__200


In [287]:
cp = CandlePlot(df_gains, candles=False)
cp.show_plot(line_traces=['GAIN_C'])